In [1]:
import sys
sys.path.append('C:/Users/dust/Documents/Github/KoDeBERTa')

In [2]:
import torch
import pytorch_lightning as pl

from tokenizers import Tokenizer
from datasets import IterableDataset
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint

from Data.DataCollator import DataCollatorForSentencePieceSpanMLM
from Model.DebertaV3 import DebertaV3ForPretraining

In [3]:
torch.set_float32_matmul_precision('medium')

In [4]:
SEED=0

In [5]:
pl.seed_everything(SEED)

Global seed set to 0


0

In [6]:
tokenizer = Tokenizer.from_file('../.local/tokenizers/mecab-hf-unigram-880M-128k.json')

In [7]:
mask_id = tokenizer.get_vocab()['[MASK]']
pad_id = tokenizer.get_vocab()['[PAD]']

In [8]:
def gen():
    with open('../../../Preproc5/korean/mecab.txt', encoding='utf-8-sig') as f:
        for line in f:
            yield line

In [9]:
batch_size = 4
max_steps = 500_000 * (8192//batch_size)

In [10]:
ds = IterableDataset.from_generator(gen)
ds.shuffle(seed=SEED, buffer_size=8_800_000)
dl = DataLoader(ds, batch_size=batch_size, collate_fn=DataCollatorForSentencePieceSpanMLM(tokenizer, truncation_argument={'max_length':512}))

In [11]:
debertav3_pretrainer = DebertaV3ForPretraining('microsoft/deberta-v3-xsmall', mask_id=mask_id, pad_id=pad_id, lr=1e-7, num_warmup_steps=int(max_steps*0.08), num_training_steps=max_steps)

In [12]:
checkpoint_callback = ModelCheckpoint(
    monitor='Loss_D',
    dirpath='../.model/',
    filename='_{epoch:02d}-{Loss_G:.4f}-{Loss_D:.4f}',
    save_weights_only=True,
)

trainer = pl.Trainer(
    accelerator='gpu',
    precision=16,
    max_steps=max_steps,
    # accumulate_grad_batches=256//batch_size,
    callbacks=[checkpoint_callback],
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(debertav3_pretrainer,dl)

c:\Users\dust\anaconda3\envs\py310_torch1\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
c:\Users\dust\anaconda3\envs\py310_torch1\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory C:\Users\dust\Documents\Github\KoDeBERTa\.model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\dust\anaconda3\envs\py310_torch1\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name          | Type                            | Params
--------------------------------

Training: 0it [00:00, ?it/s]